In [47]:
import itkConfig
itkConfig.LazyLoading = False
import itk

import sys
import numpy as np
from itkwidgets import view

In [48]:
Dimension = 3
InputPixelType = itk.F
OutputPixelType = itk.UC
ImageType = itk.Image[InputPixelType,Dimension]
UCImageType = itk.Image[OutputPixelType,Dimension]

In [49]:
inputFileName = '/work/Projects/ToothSegmentationProject/code/SegmentWithGeodesicActiveContourLevelSet/inputScan.nrrd'
image = itk.imread(inputFileName, InputPixelType)
view(image,cmap="Grayscale", ui_collapsed="True", mode="x")

Vmlld2VyKGNtYXA9dSdHcmF5c2NhbGUnLCBtb2RlPXUneCcsIHJlbmRlcmVkX2ltYWdlPTxpdGtJbWFnZVB5dGhvbi5pdGtJbWFnZUYzOyBwcm94eSBvZiA8U3dpZyBPYmplY3Qgb2YgdHlwZSDigKY=


In [50]:
# Note that source image includes the soft tissues of the cheeks and skin
view(image,cmap="Grayscale")

Vmlld2VyKGNtYXA9dSdHcmF5c2NhbGUnLCByZW5kZXJlZF9pbWFnZT08aXRrSW1hZ2VQeXRob24uaXRrSW1hZ2VGMzsgcHJveHkgb2YgPFN3aWcgT2JqZWN0IG9mIHR5cGUgJ2l0a0ltYWdlRjPigKY=


In [51]:
smoothing=itk.CurvatureAnisotropicDiffusionImageFilter(image, TimeStep=0.0125, NumberOfIterations=5, ConductanceParameter=9.0)
view(smoothing,cmap="Grayscale",mode="x")

Vmlld2VyKGNtYXA9dSdHcmF5c2NhbGUnLCBtb2RlPXUneCcsIHJlbmRlcmVkX2ltYWdlPTxpdGtJbWFnZVB5dGhvbi5pdGtJbWFnZUYzOyBwcm94eSBvZiA8U3dpZyBPYmplY3Qgb2YgdHlwZSDigKY=


In [52]:
# Compute edge image
gradientMagnitude=itk.GradientMagnitudeRecursiveGaussianImageFilter(smoothing, Sigma=0.4)
view(gradientMagnitude,ui_collapsed="True", cmap="Grayscale",mode="x")

Vmlld2VyKGNtYXA9dSdHcmF5c2NhbGUnLCBtb2RlPXUneCcsIHJlbmRlcmVkX2ltYWdlPTxpdGtJbWFnZVB5dGhvbi5pdGtJbWFnZUYzOyBwcm94eSBvZiA8U3dpZyBPYmplY3Qgb2YgdHlwZSDigKY=


In [ ]:
# Segment the bone only with a 3D region-growing algorithm
confidence_connected = itk.ConfidenceConnectedImageFilter.New(gradientMagnitude, Multiplier=2.5, 
                                                              NumberOfIterations=7, InitialNeighborhoodRadius=2,
                                                              ReplaceValue=255)
SeedType = itk.Index[Dimension]
seed1 = SeedType()

seed1[0] = 136
seed1[1] = 61
seed1[2] = 61
confidence_connected.AddSeed(seed1)

seed2 = SeedType()
seed2[0] = 195
seed2[1] = 132
seed2[2] = 87
confidence_connected.AddSeed(seed2)

seed3 = SeedType()
seed3[0] = 228
seed3[1] = 163
seed3[2] = 184
confidence_connected.AddSeed(seed3)

seed4 = SeedType()
seed4[0] = 55
seed4[1] = 154
seed4[2] = 132
confidence_connected.AddSeed(seed4)

seed5 = SeedType()
seed5[0] = 87
seed5[1] = 78
seed5[2] = 72
confidence_connected.AddSeed(seed5)

confidence_connected.Update()

invert = itk.InvertIntensityImageFilter(confidence_connected)
view(invert,ui_collapsed="True", cmap="Grayscale")